**Please set up your credentials JSON as GCP_CREDENTIALS secrets**

In [1]:
import sys
print(sys.executable)

/home/me/tmp/DataTalks/znyiri100/data-engineering-zoomcamp/03-data-warehouse/.venv/bin/python


In [38]:
import os

# from google.colab import userdata
# os.environ["DESTINATION__CREDENTIALS"] = userdata.get("GCP_CREDENTIALS")
# os.environ["BUCKET_URL"] = "gs://your_bucket_url"

# Load from .env file
from dotenv import load_dotenv
load_dotenv()

# In your .env file, add:
# DESTINATION__CREDENTIALS=<your-json-credentials-as-string>
# BUCKET_URL=gs://your_bucket_url

True

In [41]:
!echo $BUCKET_URL #$DESTINATION__CREDENTIALS

gs://zoomcamp_bucket_kestra-sandbox-8656


In [4]:
# Install for production
# %%capture
# !pip install dlt[bigquery, gs]

!uv run which python
!uv add dlt[bigquery,gs,duckdb]
!uv pip show dlt

/home/me/tmp/DataTalks/znyiri100/data-engineering-zoomcamp/03-data-warehouse/.venv/bin/python
Resolved 165 packages in 1ms
Audited 159 packages in 3ms
Name: dlt
Version: 1.21.0
Location: /home/me/tmp/DataTalks/znyiri100/data-engineering-zoomcamp/03-data-warehouse/.venv/lib/python3.11/site-packages
Requires: click, fsspec, gitpython, giturlparse, humanize, jsonpath-ng, orjson, packaging, pathvalidate, pendulum, pluggy, pytz, pyyaml, requests, requirements-parser, rich-argparse, semver, setuptools, simplejson, sqlglot, tenacity, tomlkit, typing-extensions, tzdata
Required-by:


In [5]:
# Install for testing
# %%capture
# !pip install dlt[duckdb]

In [6]:
import dlt
import requests
import pandas as pd
from dlt.destinations import filesystem
from io import BytesIO

Ingesting parquet files to GCS.

In [40]:
# Define a dlt source to download and process Parquet files as resources
@dlt.source(name="rides")
def download_parquet():
    prefix = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata"
    for month in range(1, 7):
    # for month in range(1, 3):        
        file_name = f"yellow_tripdata_2024-0{month}.parquet"
        url = f"{prefix}_2024-0{month}.parquet"
        response = requests.get(url)

        df = pd.read_parquet(BytesIO(response.content))
        print(month,df.shape, url)

        # Return the dataframe as a dlt resource for ingestion
        yield dlt.resource(df, name=file_name)


# Initialize the pipeline
pipeline = dlt.pipeline(
    pipeline_name="rides_pipeline",
    destination=filesystem(layout="{schema_name}/{table_name}.{ext}"),
    dataset_name="rides_dataset",
)

# Run the pipeline to load Parquet data into DuckDB
load_info = pipeline.run(download_parquet(), loader_file_format="parquet")

# # Print the results
print(load_info)


2026-02-07 06:17:46,532|[WARNING]|1908999|128538107901760|dlt|pipeline.py|_state_to_props:1731|The destination dlt.destinations.duckdb:None in state differs from destination dlt.destinations.filesystem:filesystem in pipeline and will be ignored


1 (2964624, 19) https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet
2 (3007526, 19) https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-02.parquet
3 (3582628, 19) https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-03.parquet
4 (3514289, 19) https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-04.parquet
5 (3723833, 19) https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-05.parquet
6 (3539193, 19) https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-06.parquet
Pipeline rides_pipeline load step completed in 30.17 seconds
1 load package(s) were loaded to destination filesystem and into dataset rides_dataset
The filesystem destination used gs://zoomcamp_bucket_kestra-sandbox-8656 location to store data
Load package 1770463097.7839444 is LOADED and contains no failed jobs


Ingesting data to Database

In [24]:
# Unset the credentials for DuckDB testing
if "DESTINATION__CREDENTIALS" in os.environ:
    del os.environ["DESTINATION__CREDENTIALS"]
    
!echo $BUCKET_URL $DESTINATION__CREDENTIALS

gs://zoomcamp_bucket_kestra-sandbox-8656


In [35]:
# Define a dlt resource to download and process Parquet files as single table
@dlt.resource(name="rides", write_disposition="replace")
def download_parquet():
    prefix = 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata'

    for month in range(1, 7):
    # for month in range(1, 2):        
        url = f"{prefix}_2024-0{month}.parquet"
        response = requests.get(url)

        df = pd.read_parquet(BytesIO(response.content))
        print(month,df.shape, url)

        yield df


# Initialize the pipeline
pipeline = dlt.pipeline(
    pipeline_name="rides_pipeline",
    destination="duckdb",  # Use DuckDB for testing
    # destination="bigquery",  # Use BigQuery for production
    dataset_name="rides_dataset",
)

# Run the pipeline to load Parquet data into DuckDB
info = pipeline.run(download_parquet)

# Print the results
print(info)


1 (2964624, 19) https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet
2 (3007526, 19) https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-02.parquet
3 (3582628, 19) https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-03.parquet
4 (3514289, 19) https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-04.parquet
5 (3723833, 19) https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-05.parquet
6 (3539193, 19) https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-06.parquet
Pipeline rides_pipeline load step completed in 10.42 seconds
1 load package(s) were loaded to destination duckdb and into dataset rides_dataset
The duckdb destination used duckdb:////home/me/tmp/DataTalks/znyiri100/data-engineering-zoomcamp/03-data-warehouse/rides_pipeline.duckdb location to store data
Load package 1770462775.2709737 is LOADED and contains no failed jobs


In [36]:
import duckdb


conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

# Set search path to the dataset
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")

# Describe the dataset to see loaded tables
res = conn.sql("DESCRIBE").df()
res

,database,schema,name,column_names,column_types,temporary
0,rides_pipeline,rides_dataset,_dlt_loads,"[load_id, schema_name, status, inserted_at, sc...","[VARCHAR, VARCHAR, BIGINT, TIMESTAMP WITH TIME...",False
1,rides_pipeline,rides_dataset,_dlt_pipeline_state,"[version, engine_version, pipeline_name, state...","[BIGINT, BIGINT, VARCHAR, VARCHAR, TIMESTAMP W...",False
2,rides_pipeline,rides_dataset,_dlt_version,"[version, engine_version, inserted_at, schema_...","[BIGINT, BIGINT, TIMESTAMP WITH TIME ZONE, VAR...",False
3,rides_pipeline,rides_dataset,rides,"[vendor_id, tpep_pickup_datetime, tpep_dropoff...","[INTEGER, TIMESTAMP WITH TIME ZONE, TIMESTAMP ...",False


In [37]:
# provide a resource name to query a table of that name
with pipeline.sql_client() as client:
    with client.execute_query(f"SELECT count(1) FROM rides") as cursor:
        data = cursor.df()
print(data)

   count(1)
0  20332093
